# 03. 수익률 심층 분석 (Revenue Analysis)

가격 전략, 수익 최적화 포인트를 분석합니다.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from models.database import session_scope
from models.schema import Station, Listing, DailyStat

In [ ]:
with session_scope() as session:
    stations = pd.read_sql(session.query(Station).statement, session.bind)
    listings = pd.read_sql(session.query(Listing).statement, session.bind)
    daily_stats = pd.read_sql(session.query(DailyStat).statement, session.bind)

daily_stats['date'] = pd.to_datetime(daily_stats['date'])
total_stats = daily_stats[daily_stats['room_type'].isna()].copy()

print(f'총 추정 수익 합계: {total_stats["estimated_revenue"].sum():,.0f} KRW')

## 1. 역별 월간 추정 수익 상위 20개

In [ ]:
merged = total_stats.merge(stations[['id', 'name']], left_on='station_id', right_on='id')

top_revenue = (
    merged.groupby('name')['estimated_revenue']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .reset_index()
)
top_revenue.columns = ['역명', '총 추정 수익']

fig = px.bar(top_revenue, x='역명', y='총 추정 수익',
             title='역별 총 추정 수익 상위 20개',
             labels={'총 추정 수익': '총 추정 수익 (KRW)'})
fig.show()

## 2. 숙소 유형별 수익성 비교

In [ ]:
room_revenue = (
    daily_stats[daily_stats['room_type'].notna()]
    .groupby('room_type')
    .agg(
        avg_booking_rate=('booking_rate', 'mean'),
        avg_price=('avg_daily_price', 'mean'),
        total_revenue=('estimated_revenue', 'sum'),
    )
    .reset_index()
)

fig = go.Figure()
fig.add_trace(go.Bar(name='평균 예약률', x=room_revenue['room_type'],
                     y=room_revenue['avg_booking_rate'], yaxis='y'))
fig.add_trace(go.Scatter(name='평균 가격', x=room_revenue['room_type'],
                          y=room_revenue['avg_price'], yaxis='y2', mode='lines+markers'))
fig.update_layout(
    title='숙소 유형별 예약률 vs 평균 가격',
    yaxis=dict(title='예약률', tickformat='.0%'),
    yaxis2=dict(title='평균 가격 (KRW)', overlaying='y', side='right'),
)
fig.show()

## 3. 수익 최적화 분석 - 가격 탄력성

In [ ]:
# 가격 구간별 예약률 분석
price_stats = total_stats[total_stats['avg_daily_price'] > 0].copy()
price_stats['price_range'] = pd.cut(
    price_stats['avg_daily_price'],
    bins=[0, 50000, 100000, 150000, 200000, 300000, float('inf')],
    labels=['~5만', '5~10만', '10~15만', '15~20만', '20~30만', '30만+'],
)

price_group = (
    price_stats.groupby('price_range', observed=True)['booking_rate']
    .mean()
    .reset_index()
)

fig = px.bar(price_group, x='price_range', y='booking_rate',
             title='가격 구간별 평균 예약률',
             labels={'price_range': '가격 구간 (KRW)', 'booking_rate': '평균 예약률'})
fig.update_yaxes(tickformat='.0%')
fig.show()

## 4. 수익 추이 - 일별

In [ ]:
daily_revenue = (
    total_stats.groupby('date')['estimated_revenue']
    .sum()
    .reset_index()
)

fig = px.area(daily_revenue, x='date', y='estimated_revenue',
              title='일별 전체 역 총 추정 수익 추이',
              labels={'date': '날짜', 'estimated_revenue': '총 추정 수익 (KRW)'})
fig.show()

## 5. 수익률 상위/하위 역 비교

In [ ]:
station_avg = (
    merged.groupby('name')
    .agg(
        avg_rate=('booking_rate', 'mean'),
        avg_price=('avg_daily_price', 'mean'),
        avg_revenue=('estimated_revenue', 'mean'),
        listings=('total_listings', 'mean'),
    )
    .reset_index()
)

print('=== 수익 상위 5개 역 ===')
print(station_avg.sort_values('avg_revenue', ascending=False).head(5).to_string(index=False))

print('\n=== 수익 하위 5개 역 (숙소 있는 역) ===')
print(
    station_avg[station_avg['listings'] > 0]
    .sort_values('avg_revenue')
    .head(5)
    .to_string(index=False)
)